<a href="https://colab.research.google.com/github/BrianvanWinden/TM10007_Group_13/blob/master/Code_opgeschoond_en_werkend_random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing functions

In [9]:
# General packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import datasets as ds
from scipy.stats import randint
from zipfile import ZipFile
from scipy import stats
from statistics import stdev


# Classifiers and kernels
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import PCA
from sklearn import preprocessing, metrics
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, roc_auc_score

# Regularization
from sklearn.linear_model import Lasso, RidgeClassifier
from sklearn.feature_selection import SelectFromModel

# Model selection
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, learning_curve, ShuffleSplit, StratifiedKFold

from sklearn.pipeline import Pipeline

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc

# Sampling
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours

from sklearn.pipeline import make_pipeline



def evaluation(y_val, y_pred, title = 'Confusion Matrix'):
    cm = confusion_matrix(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    # print('Recall: ', recall)
    # print('Accuracy: ', accuracy)
    # print('Precision: ', precision)
    # print('F1: ', f1)
    sns.heatmap(cm,  cmap= 'PuBu', annot=True, fmt='g', annot_kws=    {'size':20})
    plt.xlabel('predicted', fontsize=18)
    plt.ylabel('actual', fontsize=18)
    plt.title(title, fontsize=18)

def plot_learning_curve(estimator, title, X, y, axes, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    

    axes.set_title(title)
    if ylim is not None:
        axes.set_ylim(*ylim)
    axes.set_xlabel("Training examples")
    axes.set_ylabel("Score")

    train_sizes, train_scores, test_scores  = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes.legend(loc="best")
    return plt




def roc_plotter(fpr, tpr, roc_auc):
    """ Plot the ROC curve of the classifier for the given number of components

    - param numpy.array fpr: False positive rate
    - param numpy.array tpr: True positive rate
    - param integer roc_auc: Area under the ROC curve
    """
    plt.plot(fpr, tpr, linewidth=2, label=f'ROC curve (area = {roc_auc:0.2f})')

    # Properties ROC curve figure
    plt.plot([0, 1], [0, 1], color='navy', linewidth=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")




# Assignment arrhythmia classification

In [29]:
# Run this to use from colab environment
!git clone https://github.com/BrianvanWinden/TM10007_Group_13.git
!unzip "/content/TM10007_Group_13/ecg/ecg_data.zip"
import pandas as pd 

data = pd.read_csv("ecg_data.csv") 

data_points= data.drop(['label', 'Unnamed: 0'], axis=1).to_numpy()
data_labels= data['label'].to_numpy()


x_train_val, x_test, y_train_val, y_test = train_test_split(data_points, data_labels, test_size=0.3, stratify=data_labels)



# Taking a look at the data

In [31]:
# Samples and features
print(data.head)
print(f'The number of samples: {len(data.index)}')
print(f'The number of features: {len(data.columns)}')
healthy_patients = (list(data['label'] == 0)).count(True)
print(f'The number of healthy patients: {healthy_patients}') 
sick_patients = (list(data['label'] == 1)).count(True)
print(f'The number of sick patients: {sick_patients}')
percentage_sick=sick_patients/(sick_patients+healthy_patients)
print(f'Percentage of sick patients: {round(percentage_sick,2)*100}%')

# Outliers # aanpassen
z = np.abs(stats.zscore(data))
x = np.where(z>3)
print(f'Number of datapoints where the Z score is larger than 3: {len(x[1])}')

# Missing values
missing_values = data.isna().sum()
number_missing_values = missing_values.astype(bool).sum(axis=0)
print(f'Number of missig data points:{number_missing_values}')




<bound method NDFrame.head of      Unnamed: 0         0_0         0_1  ...    11_748    11_749  label
0             0   47.685046   48.416904  ...  0.269249  0.112248      0
1             1  152.726718  297.353726  ...  0.859936  0.742673      1
2             2    1.601260    3.882169  ...  0.098506  0.098288      0
3             3    1.388947    3.052483  ...  0.092000  0.192559      0
4             4    3.625561    3.728466  ...  0.141610  0.129477      0
..          ...         ...         ...  ...       ...       ...    ...
822         822   14.673713   13.879148  ...  0.155580  0.061931      0
823         823    3.167367   13.771749  ...  0.771741  0.757218      0
824         824    0.439357    8.700374  ...  0.159615  0.077114      0
825         825    0.235515    4.074581  ...  0.060548  0.116350      0
826         826    0.301328    2.012000  ...  0.114943  0.223285      1

[827 rows x 9002 columns]>
The number of samples: 827
The number of features: 9002
The number of healthy 

# Preprocessing and classifier validation

In [ ]:
# Splitting data  
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

cv_outer = StratifiedKFold(n_splits=5, shuffle=True)
cv_inner = StratifiedKFold(n_splits=3, shuffle=True)
outer_results = list()
all_scores = list()
models = list()

for train_ix, val_ix in cv_outer.split(x_train_val,y_train_val):
  # split data
  X_train, X_val = x_train_val[train_ix, :], x_train_val[val_ix, :]
  y_train, y_val = y_train_val[train_ix], y_train_val[val_ix]
  print (X_train.shape)
  print(X_val.shape)
  #Resampling
  # combines SMOTE over and edited nearest neighbour undersampling
  # edited nearest neighbours kan van beide groepen weghalen, met sampling_strategy='majority' alleen van grote groep
  # the authors comment that ENN is more aggressive at downsampling the majority class than Tomek Links, providing more in-depth cleaning. They apply the method, removing examples from both the majority and minority classes.
  #A Study of the Behavior of Several Methods for Balancing Machine Learning Training Data, 2004.
  # resample = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))
  # X_resampled, y_resampled = resample.fit_resample(X_train, y_train)
  X_resampled, y_resampled = X_train, y_train

  #Scaling
  scaler = preprocessing.RobustScaler()
  scaler.fit(X_resampled)
  x_train_scaled = scaler.transform(X_resampled)
  x_val_scaled = scaler.transform(X_val)

  #PCA
  pca = PCA(n_components=0.99)
  pca = pca.fit(x_train_scaled)
  x_train_trans = pca.transform(x_train_scaled)
  x_val_trans = pca.transform(x_val_scaled)
  print(f'Amount of features from PCA: {x_train_trans.shape[1]}')
  model_params = {
                  'svm_rbf': 
                  {'model': SVC(kernel='rbf', class_weight='balanced'),
                    'params': {'C': np.arange(0.01,100,0.01),
                               'gamma': np.arange(0.01,100,0.01)
                               }
                  },
                   'svm_lin': 
                  {'model': SVC(kernel='linear', class_weight='balanced'),
                    'params': {'C': np.arange(0.01,100,0.01)
                               }
                   },
                  'linsvc': 
                  {'model': LinearSVC(class_weight='balanced', dual=False, max_iter=10000),
                   'params':{'C': np.arange(0.01,100,0.01)
                             }
                  },
                  'logreg':
                  {'model': LogisticRegression(class_weight='balanced', dual=False, max_iter=10000),
                   'params':{'C': np.arange(0.01,100,0.01)
                             }                      
                  # # },
                  # # 'lda': 
                  # # {'model': LinearDiscriminantAnalysis(),
                  # # 'params':{}
                  # # },
                  # # 'knn':
                  # # {'model': KNeighborsClassifier(weights='distance'),
                  # #  'params':{'n_neighbors': [5,10,15]
                  # #            }
                   }
                }

  for model_name, mp in model_params.items():
    search = RandomizedSearchCV(mp['model'], mp['params'], cv = cv_inner, scoring = 'roc_auc', return_train_score=True)
    result = search.fit(x_train_scaled, y_resampled)
    best_model = result.best_estimator_
    models.append(best_model)
    yhat = best_model.predict(x_val_scaled)
    auc_score_val = roc_auc_score(y_val, yhat)
    outer_results.append(auc_score_val)
    all_scores.append(result.best_score_)
    print(f'auc: {auc_score_val}, result.best_score_:{result.best_score_}, best parameters:{result.best_params_}')



(462, 9001)
(116, 9001)
Amount of features from PCA: 278
auc: 0.5, result.best_score_:0.5, best parameters:{'gamma': 60.72, 'C': 36.62}
auc: 0.7541666666666668, result.best_score_:0.8150315535954831, best parameters:{'C': 71.7}


# Best classifier for test set

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

models_df = pd.DataFrame(models)
outer_results_df = pd.DataFrame(outer_results)
all_scores_df = pd.DataFrame(all_scores)
results = pd.concat([models_df, outer_results_df, all_scores_df], axis=1)
results.columns = ['Models', 'Auc score', 'Best score']
results_sorted = results.sort_values(by=['Auc score'], ascending=False)
print(results_sorted)

best_classifier = results_sorted['Models'].iloc[0]
print(f'The best classifier is: {best_classifier}')

# cv_test = StratifiedKFold(n_splits=2, shuffle=True)
# # ('resampling', SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))),('scaling', preprocessing.RobustScaler()),('pca', PCA(n_components=0.99)),
# clf = Pipeline([('pca', PCA(n_components=0.99)), ('clf', best_classifier)])
# cross_score = cross_val_score(clf, x_test, y_test, cv=cv_test, scoring='roc_auc')
# print(f'Mean score of classifier on test data: {round(mean(cross_score),3)} + {round(stdev(cross_score),3)}')

#Resampling
resample = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))
X_resampled, y_resampled = resample.fit_resample(x_train_val, y_train_val)

#Scaling
scaler = preprocessing.RobustScaler()
scaler.fit(X_resampled)
x_train_scaled = scaler.transform(X_resampled)
x_test_scaled = scaler.transform(x_test)


#PCA
pca = PCA(n_components=0.99)
pca = pca.fit(x_train_scaled)
x_train_trans = pca.transform(x_train_scaled)
x_test_trans = pca.transform(x_test_scaled)

# Evaluation classifier
clf = best_classifier.fit(x_train_trans, y_resampled)
y_pred = clf.predict(x_test_trans)
y_score = clf.decision_function(x_test_trans)
accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
auc_score = metrics.roc_auc_score(y_test, y_pred)
print('Misclassified: %d / %d' % ((y_test != y_pred).sum(), x_test_trans.shape[0]), str(clf))
evaluation(y_test, y_pred)

print(f'Auc score test data: {auc_score}')






# ROC curve

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

roc_plotter(fpr, tpr, roc_auc)


# Learning curves

In [ ]:
# Learning Curve

X = data_points
Y = data_labels

clsfs = {'linSVC': LinearSVC(class_weight='balanced', dual=False, max_iter=10000), 
          'SVMrbf':SVC(kernel='rbf', class_weight='balanced'), 
          'SVMlin': SVC(kernel='linear', class_weight='balanced'), 
          'Logreg': LogisticRegression(class_weight='balanced', dual=False, max_iter=10000)}

num = 0
fig = plt.figure(figsize=(24,8*len(clsfs)))
ax = fig.add_subplot(7, 3, num + 1)
ax.scatter(X[:, 0], X[:, 1], marker='o', c=Y, 
           s=25, edgecolor='k', cmap=plt.cm.Paired)

cv = ShuffleSplit(n_splits=2, test_size=0.2, random_state=0)
num = 1
for key, clf in clsfs.items():
    title = str(key)
    ax = fig.add_subplot(7, 3, num + 1)
    plot_learning_curve(clf, title, X, Y, ax, ylim=(0.3, 1.01), cv=cv)
    num += 1

# ROC curve
